In [16]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [17]:
#removing the "R-S" from location column

import pandas as pd


df = pd.read_csv("houserentdhaka_new.csv")########no lat lon

df["Location"] = df["Location"].map(lambda i:i[0:].replace(" R-A",""))

df.head()


,id,Location,Area,Bed,Bath,Price
0,0,"Block H, Bashundhara, Dhaka","1,600 sqft",3,3,23 Thousand
1,1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,23 Thousand
2,2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,22 Thousand
3,3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,78 Thousand
4,4,"Baridhara, Dhaka","2,200 sqft",3,3,77 Thousand


In [18]:
#Checking for known and unknown GeoCodes


import pandas as pd
import requests
import urllib.parse
import time

headers = {
    "User-Agent": "UniversityProject/1.0 (sidratul.montaha@g.bracu.ac.bd)"
}
# def get_geo(address):
#     address = address
#     url = f"https://nominatim.openstreetmap.org/search?q={urllib.parse.quote(address)}&format=json"
#     print (url)
#     response = requests.get(url).json()
#     print(00)
#     return response
def get_geo(address):
    url = f"https://nominatim.openstreetmap.org/search?q={urllib.parse.quote(address)}&format=json"
    print(f"Generated URL: {url}")
    
    try:
        response = requests.get(url, headers=headers)
       # print(f"Response Status Code: {response.status_code}")
        
        if response.status_code != 200:
           # print(f"Error: Received status code {response.status_code} for address {address}")
            return []

        return response.json()
    except requests.exceptions.RequestException as e:
        #print(f"Network error for address {address}: {e}")
        return []
    except ValueError as e:
       # print(f"Error decoding JSON for address {address}: {e}")
        return []


locations = df["Location"]


known = []
unknown = []
for i in range(len(locations)):
    
    resp = get_geo(locations[i])
    time.sleep(1)
    
    if len(resp)==0:
        print("Unknown",i,locations[i])
        tup = (i,locations[i])
        unknown.append(tup)
    else:
        print("Known",i,locations[i])
        lat = resp[0]["lat"]
        long = resp[0]["lon"]
        
        tup = (i,lat,long)
        known.append(tup)
print(len(known))
print(len(unknown))



Generated URL: https://nominatim.openstreetmap.org/search?q=Block%20H%2C%20Bashundhara%2C%20Dhaka&format=json
Known 0 Block H, Bashundhara, Dhaka
Generated URL: https://nominatim.openstreetmap.org/search?q=Farmgate%2C%20Tejgaon%2C%20Dhaka&format=json
Known 1 Farmgate, Tejgaon, Dhaka
Generated URL: https://nominatim.openstreetmap.org/search?q=Block%20B%2C%20Nobodoy%20Housing%20Society%2C%20Mohammadpur%2C%20Dhaka&format=json
Unknown 2 Block B, Nobodoy Housing Society, Mohammadpur, Dhaka
Generated URL: https://nominatim.openstreetmap.org/search?q=Gulshan%201%2C%20Gulshan%2C%20Dhaka&format=json
Known 3 Gulshan 1, Gulshan, Dhaka
Generated URL: https://nominatim.openstreetmap.org/search?q=Baridhara%2C%20Dhaka&format=json
Known 4 Baridhara, Dhaka
Generated URL: https://nominatim.openstreetmap.org/search?q=Bashundhara%2C%20Dhaka&format=json
Known 5 Bashundhara, Dhaka
Generated URL: https://nominatim.openstreetmap.org/search?q=Baridhara%2C%20Dhaka&format=json
Known 6 Baridhara, Dhaka
Generated 

In [4]:
#GeoCode

import requests
import urllib.parse
headers = {
    "User-Agent": "UniversityProject/1.0 (sidratul.montaha@g.bracu.ac.bd)"
}
address = '60 feet pirerbag'
url = f"https://nominatim.openstreetmap.org/search?q={urllib.parse.quote(address)}&format=json"
print(url)
response = requests.get(url, headers=headers)
print(response.json())
print(response[0]["lat"])
print(response[0]["lon"])

https://nominatim.openstreetmap.org/search?q=60%20feet%20pirerbag&format=json
[{'place_id': 230155278, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 711271207, 'lat': '23.78301305', 'lon': '90.36844305', 'class': 'highway', 'type': 'tertiary', 'place_rank': 26, 'importance': 0.05340434500648505, 'addresstype': 'road', 'name': '৬০ ফুট', 'display_name': '৬০ ফুট, আমতলা, পীরেরবাগ, ঢাকা, ঢাকা মহানগর, ঢাকা জেলা, ঢাকা বিভাগ, 1916, বাংলাদেশ', 'boundingbox': ['23.7829188', '23.7831073', '90.3684110', '90.3684751']}]


TypeError: 'Response' object is not subscriptable

In [19]:
#Ration of latidude and logntude
new_df = pd.read_csv("houses.csv")
new_df.head()

pd.options.mode.chained_assignment = None  # Default='warn'
new_df.isnull().sum()
for tup in known:
    ratio = float(tup[1])/float(tup[2])
    
    new_df["lat-lon"][tup[0]] = ratio

new_df.head()


/var/folders/74/k_ffvg497wdg1nn73q0nzhq40000gn/T/ipykernel_9992/1355302098.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df["lat-lon"][tup[0]] = ratio
/var/folders/74/k_ffvg497wdg1nn73q0nzhq40000gn/T/ipykernel_9992/1355302098.py:1

,id,Location,lat-lon,Area,Bed,Bath,Price
0,0,"Block H, Bashundhara R-A, Dhaka",0.263433,"1,600 sqft",3,3,23 Thousand
1,1,"Farmgate, Tejgaon, Dhaka",0.262859,900 sqft,2,2,23 Thousand
2,2,"Block B, Nobodoy Housing Society, Mohammadpur,...",NaN,"1,250 sqft",3,3,22 Thousand
3,3,"Gulshan 1, Gulshan, Dhaka",0.263010,"2,200 sqft",3,4,78 Thousand
4,4,"Baridhara, Dhaka",0.263252,"2,200 sqft",3,3,77 Thousand


In [20]:
#new_df.isnull().sum()
new_df = new_df.dropna()
new_df.isnull().sum()

id          0
Location    0
lat-lon     0
Area        0
Bed         0
Bath        0
Price       0
dtype: int64

In [21]:
new_df.head()
new_df.drop(["id"],axis=1)

,Location,lat-lon,Area,Bed,Bath,Price
0,"Block H, Bashundhara R-A, Dhaka",0.263433,"1,600 sqft",3,3,23 Thousand
1,"Farmgate, Tejgaon, Dhaka",0.262859,900 sqft,2,2,23 Thousand
3,"Gulshan 1, Gulshan, Dhaka",0.263010,"2,200 sqft",3,4,78 Thousand
4,"Baridhara, Dhaka",0.263252,"2,200 sqft",3,3,77 Thousand
5,"Bashundhara R-A, Dhaka",0.263335,"3,000 sqft",4,5,52 Thousand
...,...,...,...,...,...,...
494,"Sector 14, Uttara, Dhaka",0.264048,900 sqft,3,2,22 Thousand
495,"Gulshan 1, Gulshan, Dhaka",0.263010,"2,100 sqft",3,3,64 Thousand
496,"Dakkhin Paikpara, Paikpara, Mirpur, Dhaka",0.263403,"1,100 sqft",3,2,18 Thousand
497,"Dhanmondi, Dhaka",0.262847,"1,600 sqft",3,4,38 Thousand


In [22]:
new_df.head()
new_df.drop(["id"],axis=1)
new_df.to_csv('new_house_data.csv', encoding='utf-8')######new